In [1]:
import pandas as pd
from nltk.corpus import stopwords
from textblob import Word
import tweepy, pickle

sw=stopwords.words("english")

In [2]:
def data_cleaning(df,choice):
    df = df.applymap(lambda s: s.lower() if type(s) == str else s) # Tüm metinler küçük harf formatına dönüştürülür
    df["text"]=df["text"].str.replace("[^\w\s]","") # noktalama işaretlerinin silinmesi
    df["text"]=df["text"].str.replace("\d","") # sayıların silinmesi
    df["text"]=df["text"].apply(lambda x: " ".join(x for x in x.split() if x not in sw)) # stopwords kelimelerin silinmesi
    if choice == 1:
        df["text"]=df["text"].apply(lambda x: " ".join(Word(i).lemmatize() for i in x.split())) # lemmatization
    elif choice == 2:
        df["text"]=df["text"].apply(lambda x: " ".join(Word(i).stem() for i in x.split())) # stemming
    return df['text']

In [3]:
def data_cleaning_text(text,choice):
    text=re.sub(r'[^\w\s]', '', text)
    text=re.sub(r'[0-9]+', '', text)
    text=' '.join(x for x in text.split() if x not in sw)
    if choice == 1:
        text = " ".join(Word(i).lemmatize() for i in text.split())
    elif choice == 2:
        text = " ".join(Word(i).stem() for i in text.split())
    return text.lower()

In [18]:
def hashtag_df(key, count):
    consumer_key = 'CONSUMER'
    consumer_secret = 'CONSUMER_SECRET'
    access_token = 'TOKEN'
    access_token_secret = 'TOKEN_SECRET'
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    API = tweepy.API(auth)
    tweetler=API.search_tweets(q=key, lang="en", result_type="mixed", count=count)
    df = pd.DataFrame()
    df["text"] = [tweet.text for tweet in tweetler]
    return df

In [27]:
key = input('Input a key:')

Input a key:yemeksepeti


In [28]:
count = input('Input a count value:')

Input a count value:100


In [29]:
df = hashtag_df(key, count)

In [30]:
df

,text
0,"🇫🇷 Fransa tarihinin en golcü oyuncusu kimdir, ..."
1,🔥 İngiltere-Fransa maçı kaç kaç biter?\n\n🎉 Do...
2,Yemeksepeti’nin kurucusu Nevzat Aydın’ın forma...
3,@TheGokturkFB @EnSuperLig @yemeksepeti Fenerba...
4,@yemeksepeti Sipariş tarihini bilerek mi sildi...
...,...
95,✅ Messi yanıtını vererek @yemeksepeti'nden 100...
96,✅ Ronaldo yanıtını vererek @yemeksepeti'nden 1...
97,"@yemeksepeti bir sipariş 1,5 saatte nasıl gelm..."
98,@EnSuperLig @yemeksepeti Bu erden timur agladi...


In [9]:
df.text = data_cleaning(df,1)

C:\Users\OUZHAN~1\AppData\Local\Temp/ipykernel_5884/2298853171.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text"]=df["text"].str.replace("[^\w\s]","") # noktalama işaretlerinin silinmesi
C:\Users\OUZHAN~1\AppData\Local\Temp/ipykernel_5884/2298853171.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text"]=df["text"].str.replace("\d","") # sayıların silinmesi


In [10]:
linear = pickle.load(open('logistic.sav', 'rb'))
ridge = pickle.load(open('ridge.sav', 'rb'))
perceptron = pickle.load(open('perceptron.sav', 'rb'))
xgb = pickle.load(open('xgb.sav', 'rb'))
lgbm = pickle.load(open('lgbm.sav', 'rb'))
vectorizer = pickle.load(open('tfidf_vectorizer.sav', 'rb'))

In [23]:
test_data = vectorizer.transform(df.text)

In [24]:
print('Logistic Regression Classifier Prediction:',linear.predict(test_data))
print('Ridge Classifier Prediction:',ridge.predict(test_data))
print('Perceptron Classifier Prediction:',perceptron.predict(test_data))
print('XGBoost Classifier Prediction:',xgb.predict(test_data))
print('LightGBM Classifier Prediction:',lgbm.predict(test_data))

Logistic Regression Classifier Prediction: [0 1 2 1 1 1 0 1 0 1 1 2 2 0 1 1 1 0 0 1 0 1 1 0 1 1 2 1 0 1 1 2 2 1 1 1 1
 0 0 0 0 1 1 1 1 1 0 1 0 1 0 0 1 1 0 0 0 0 1 1 1 0 0 1 1 2 1 1 1 1 0 0 0 1
 2 2 1 0 0 1 0 0 1 0 0 1 1 0 1 0 1 0 0 0 0 1 0 1 2 0]
Ridge Classifier Prediction: [0 1 2 1 1 1 0 1 1 1 1 2 2 0 1 1 1 0 0 1 0 1 1 0 0 1 2 1 0 1 1 2 2 1 1 1 1
 0 0 0 0 1 1 1 1 1 1 1 0 1 0 0 1 0 0 0 0 0 1 1 1 0 0 1 0 2 1 1 1 1 0 0 0 1
 2 2 1 0 0 1 1 1 0 0 0 1 1 0 1 0 1 0 0 0 0 1 0 1 2 0]
Perceptron Classifier Prediction: [0 1 2 1 1 1 0 2 0 1 1 2 2 0 0 2 1 0 0 0 0 1 1 0 1 1 2 0 0 1 1 2 2 1 1 1 1
 0 0 0 0 1 2 1 1 1 2 0 0 0 0 0 0 2 2 0 2 2 0 0 1 0 0 1 0 2 0 0 0 1 2 0 0 1
 2 2 1 0 0 0 0 0 0 0 2 0 0 0 1 0 1 0 0 2 0 0 0 0 2 0]
XGBoost Classifier Prediction: [0 0 2 1 1 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1 2 0 0 1 0 2 2 0 0 1 1
 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 2 0 0 0 1 0 0 0 1
 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 2 0]
LightGBM Classifier Prediction: [0 0 2 1 1 1 0 